In [ ]:

import numpy as np
import cv2 as cv2
import math
import fingerprint_enhancer
from skimage.morphology import skeletonize
from scipy.spatial import distance
from collections import Counter
import os

def segmentation(original_img,block_size=15):
    
    shp = original_img.shape
    segmentation_mask = np.ones(shp)
    global_threshold = np.var(original_img,axis=None)*0.1
    row,col=original_img.shape[0],original_img.shape[1]
    i = 0
    j = 0
    while i<row:
        while j<col:
            a = i+block_size
            b = j+block_size
            y = min(col,b)
            x = min(row,a)
            local_grayscale_variance = np.var(original_img[i: x, j: y])
            if local_grayscale_variance <= global_threshold:
                segmentation_mask[i: x, j: y] = 0
            j+=block_size
        i+=block_size
    return segmentation_mask
    # print(original_img.shape[0])
    # print(original_img.shape[1])
    # print(global_threshold)

def erosion(img,kernel):
    ero = cv2.erode(img,kernel,iterations = 1)
    return ero

def dilation(img,kernel):
    dil = cv2.dilate(img,kernel,iterations = 1)
    return dil

def seg_masking(original_img,segmentation_mask,block_size=15):
    # Apply Segmentation mask
    segmented_image = original_img.copy()
    # Removing unnecessary noise in the image
    doub = 2*block_size
    kernel_open_close = cv2.getStructuringElement(cv2.MORPH_RECT,(doub,doub))
    # segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_CLOSE, kernel_open_close)
    # segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_OPEN, kernel_open_close)
    # Opening is the operation of erosion followed by dilation to remove noise
    segmentation_mask = erosion(segmentation_mask,kernel_open_close)   
    segmentation_mask = dilation(segmentation_mask,kernel_open_close)
    # Closing Dilation is the operation of followed by Erosion to remove noise
    segmentation_mask = dilation(segmentation_mask,kernel_open_close)
    segmentation_mask = erosion(segmentation_mask,kernel_open_close)
    #print(segmented_image)
    row,col=segmentation_mask.shape[0],segmentation_mask.shape[1]
    for i in range(row):
        for j in range(col):
            if segmentation_mask[i][j]==0:
                segmented_image[i][j]=255

    return segmented_image

def normalization(segmented_image):
    desired_mean = 128.0
    desired_variance = 7500.0
    current_variance,current_mean = np.var(segmented_image),np.mean(segmented_image)
    row,col=segmented_image.shape[0],segmented_image.shape[1]
    normalized_image = np.empty([row,col],dtype=float)
    for i in range(row):
        for j in range(col):
            aux=(math.sqrt(math.pow(segmented_image[i][j]-current_mean,2)*(desired_variance/current_variance)))
            if segmented_image[i][j]>current_mean:      
                normalized_image[i][j] = desired_mean+aux 
            else:
                normalized_image[i][j] = desired_mean-aux
    return normalized_image

def enhancement(normalized_image):
    enhanced_image = fingerprint_enhancer.enhance_Fingerprint(normalized_image)
    return enhanced_image

def thinning(enhanced_image):
    thinned_image = np.where(skeletonize(enhanced_image/255), 0.0, 1.0)
    return thinned_image

def get_line_ends(i, j, W, tangent):
    temp=W/2
    
    if -1 <= tangent and tangent <= 1:
        start = (int((temp) * tangent + j + temp), i)
        end = (int((temp) * tangent + j +temp), i+W)
    else:
        start = (j + math.floor(temp), int(i + temp + W/(2 * tangent)))
        end = (j - math.floor(temp), int(i + temp - W/(2 * tangent)))
    return (start, end)

def ridge_orientation_field(original_img,normalized_image,block_size=15):
    scale,delta=1,0
    grad_x = cv2.Sobel(normalized_image/255, cv2.CV_64F, 0, 1, ksize=3,scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    grad_y = cv2.Sobel(normalized_image/255, cv2.CV_64F, 1, 0, ksize=3,scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    local_directions_x = np.zeros(original_img.shape)
    local_directions_y = local_directions_x
    row,col=original_img.shape[0],original_img.shape[1]
    for i in range(0,row,1):
        for j in range(0,col,1):
            temp = block_size//2
            start_j = max(0, j-temp)
            end_j = min(j+temp,col)
            end_i = min(i+temp,row)
            start_i = max(0,i-temp)
            y = grad_y[start_i: end_i, start_j: end_j]
            x = grad_x[start_i: end_i, start_j: end_j]
            G_a = x**2-y**2
            local_directions_y[i, j] = np.sum(G_a)
            G_b = 2*x*y
            local_directions_x[i, j] = np.sum(G_b)

    siz = 2*block_size+1
    kernel_size = (siz,siz)
    gaussian_directions_x = cv2.GaussianBlur(local_directions_x, kernel_size, 1.0)
    gaussian_directions_y = cv2.GaussianBlur(local_directions_y, kernel_size, 1.0)

    orientation_map = 0.5*(np.arctan2(gaussian_directions_x, gaussian_directions_y))+0.5*np.pi

    orientation_image = cv2.cvtColor((normalized_image).astype(np.uint8), cv2.COLOR_GRAY2RGB)
    

    return orientation_image,orientation_map

#Feature extraction using crossing number technique
def crossing_number(i, j, img):
    if img[i, j] != 0.0:
        return 2.0
    else:
        row_a = [(-1, -1), (-1, 0), (-1, 1)]
        row_b = [  (0, 1),  (1, 1), (1, 0)]
        row_c = [(1, -1), (0, -1), (-1, -1)]
        offsets = row_a + row_b + row_c
        pixel_values=[]
        for x,y in offsets:
            pixel_values.append(img[i+x,j+y])
        a=0
        sum_cn = 0.0
        while a<8:
            sum_cn += abs(pixel_values[a] - pixel_values[a+1])
            a+=1

        return sum_cn // 2

# False minutiae removal close to the boundary case
def minutiae_removal(original_img,thinned_image,block_size=15):
    segment_mask_min = np.ones(original_img.shape)
    global_greyscale_variance = np.var(thinned_image)*0.1
    i,j=0,0
    row,col=original_img.shape[0],original_img.shape[1]
    while i<row:
        while j<col:
            siz_a = i+block_size
            end_i = min(row,siz_a)
            siz_b = j+block_size
            end_j = min(col,siz_b)
            local_grayscale_variance = np.var(thinned_image[i: end_i, j: end_j])
            test = lambda x : True if (local_grayscale_variance > global_greyscale_variance) else False
            if test:
                segment_mask_min[i: end_i, j: end_j] = 1
            else:
                segment_mask_min[i: end_i, j: end_j] = 0.0
            j+=block_size
        i+=block_size        

    return segment_mask_min

# Removing outermost strip 
def remove_outer_strip(original_img,minutiae,segment_mask_min,block_size=15):

    i=0
    j=0
    row,col=original_img.shape[0],original_img.shape[1]

    while j<col:
        start = 0
        siz = j+block_size
        end_j = min(col, siz)
        segment_mask_min[start: block_size, j: end_j] = 0.0
        segment_mask_min[ row-block_size:  row, j:end_j] = 0.0
        j+=block_size

    while i<row:
        start = 0
        temp1=i+block_size
        end_i = min( row, temp1)
        segment_mask_min[i: end_i, start: block_size] = 0.0
        segment_mask_min[i: end_i,  col-block_size:  col] = 0.0
        i+=block_size

    new_minutiae = {}
    neighbourhood = [(0, 1), (0, -1), (0, 0), (1, 0), (-1, 0)]
    for j in minutiae:
        x=j[0]
        y=j[1]
        
        to_append = False
        for i in range(5):
            direction_x,direction_y=neighbourhood[i][0]*block_size,neighbourhood[i][1]*block_size
            try:
                var_x = x + direction_x 
                var_y = y + direction_y
                if segment_mask_min[var_x,var_y] == 0.0:
                    to_append = True
                    break
            except IndexError:
                to_append = True
                break
        if to_append==False:
            new_minutiae[(x, y)] =  minutiae[(x, y)]
    
    return new_minutiae

# Removal of cluster
def Remove_cluster(minutiae,block_size=15):
    minutiae_list = list(minutiae.items())
    dist_thresh = block_size/2
    cluster_found = False
    cluster_list = set()
    i=1
    while i<len(minutiae_list):
        j=0
        while j<i:
            (x2, y2), (_, _) = minutiae_list[j]
            aux1=0
            (x1, y1), (_, _) = minutiae_list[i]
            if distance.euclidean((x1, y1), (x2, y2)) <= dist_thresh:
                cluster_list.add((x1, y1))
                cluster_list.add((x2, y2))
                cluster_found = True
            j+=1
        i+=1        
                    
                
    if  cluster_found==False:
        return False,minutiae
    j=0
    while j<10:
        i=0
        while i<len(minutiae_list):
            if (x1, y1) not in cluster_list:
                for (x2, y2) in cluster_list:
                    aux2=0
                    (x1, y1), _ = minutiae_list[i]
                    if distance.euclidean((x1, y1), (x2, y2))<=dist_thresh:
                        cluster_list.add((x1, y1))
            i+=1
        j+=1    
                        
    for (x1, y1) in cluster_list:
        del  minutiae[(x1, y1)]

    return True,minutiae

def fingerprint_recognition(path_fp_img):
    original_img = cv2.imread(path_fp_img, cv2.IMREAD_GRAYSCALE)
    segmentation_mask = segmentation(original_img)
    block_size = 15
    # cv2.imshow("seg_mask", segmentation_mask)
    # cv2.waitKey(0)
    segmented_image = seg_masking(original_img,segmentation_mask)
    # cv2.imshow("seg_image", segmented_image)
    # cv2.waitKey(0)
    normalized_image = normalization(segmented_image)
    # cv2.imshow("norm_image", normalized_image)
    # cv2.waitKey(0)
    enhanced_image = enhancement(normalized_image)
    # cv2.imshow("enh_image", enhanced_image)
    # cv2.waitKey(0)
    thinned_image = thinning(enhanced_image)
    # cv2.imshow("thin_image", thinned_image)
    # cv2.waitKey(0)
    orientation,map = ridge_orientation_field(original_img,normalized_image)
    # cv2.imshow("orient_imag",orientation)
    # cv2.waitKey(0)

    minutiae = {}
    minutiae_img = cv2.cvtColor((255*thinned_image).astype(np.uint8), cv2.COLOR_GRAY2RGB)
    row,col=original_img.shape[0],original_img.shape[1]
    i=1
    while i<row-1:
        j=1
        while j<col-1:
            current_cn = crossing_number(i, j,thinned_image)
            if current_cn == 1:
                minutiae[(i, j)] = (current_cn,map[i, j]) 
            elif current_cn == 3:
                minutiae[(i, j)] = (current_cn,map[i, j])
            j+=1
        i+=1        
                
    segment_mask_min = minutiae_removal(original_img,thinned_image)
    # cv2.imshow("minuate_removal",segment_mask_min)
    # cv2.waitKey(0)

    doub = 2*block_size
    kernel_open_close = cv2.getStructuringElement(cv2.MORPH_RECT,(doub,doub))
    segment_mask_min = erosion(segment_mask_min,kernel_open_close)   
    segment_mask_min = dilation(segment_mask_min,kernel_open_close)
    segment_mask_min = dilation(segment_mask_min,kernel_open_close)
    segment_mask_min = erosion(segment_mask_min,kernel_open_close)

    new_minutiae = remove_outer_strip(original_img,minutiae,segment_mask_min)
    # print(new_minutiae)

    cluster_remain = True
    while cluster_remain:
        cluster_remain,new_minutiae = Remove_cluster(new_minutiae)
    # Draw minutiae on image
    for (x, y) in new_minutiae:
        c_n, _ = new_minutiae[(x, y)]
        #if ridge end color pink them
        temp=[1,3]
        if c_n == temp[0] :
            cv2.circle(minutiae_img, (y,x), radius=3, color=(255 ,0, 255), thickness=2)
        #if ridge bifurication color green them    
        if c_n == temp[1]:
            cv2.circle(minutiae_img, (y,x), radius=3, color=(0, 255, 0), thickness=2)
    #cv2.imshow("minutiae_cluster_rem",minutiae_img)
    #cv2.waitKey(0)
    return minutiae_img,new_minutiae

def round5(x, base=5):
    return base * round(x/base)

def alignment_using_hough_transform(first, second, block_size=15):
    # Generalized Hough Transform
    # It is assumed both fingerprint have same size
    accumulator = dict()

    for (x_first, y_first)  in second.keys():
        (_, theta_t)=second[(x_first, y_first) ]
        for (x_second, y_second) in first.keys():
            (_, theta_q)=first[(x_second, y_second)]
            d_theta = theta_t - theta_q
            d_theta = min(d_theta, 2*np.pi - d_theta)
            cos_dtheta,sin_dtheta=math.cos(d_theta),math.sin(d_theta)
            d_x = x_first - x_second*cos_dtheta + y_second*sin_dtheta
            d_y = y_first - x_second*sin_dtheta - y_second*cos_dtheta
            lis=[]
            lis.append(round5(180*d_theta/np.pi, 5))
            lis.append(round5(d_x, block_size//4))
            lis.append(round5(d_y, block_size//4))
            conf=tuple(lis)
            if  accumulator.get(conf):
                accumulator[conf] =accumulator[conf]+ 1
                
            else:
                accumulator[conf] =1
    temp = max(accumulator, key=accumulator.get)
    theta=temp[0]
    x=temp[1]
    y=temp[2]
    
    return np.pi*theta/180, x, y

def pairing(first, second, transform_config,block_size=15):
    # Calculate which minutiae match to which minutiae in other fingerprint
    # Looser condition for matched minutiae than that during alignment
    flag_first,flag_second = np.zeros(len(first),), np.zeros(len(second),)
    angle_thresh = 10 * np.pi / 180
    distance_thresh = block_size/2
    count_matched = 0
    matched_minutiae = []

    i = 0
    ht_theta, _, _ = transform_config
    _,ht_x,_ = transform_config
    _,_,ht_y = transform_config
    for (x_first, y_first) in second.keys():
        j = 0
        (_, theta_t) = second[(x_first, y_first)]
        for (x_second, y_second) in first.keys():
            (_, theta_q) = first[(x_second, y_second)]
            d_theta = theta_t - theta_q - ht_theta
            d_theta = min(d_theta, 2*np.pi - d_theta)
            cos_theta,sin_theta=math.cos(ht_theta),math.sin(ht_theta)
            x_y_first = [x_first,y_first]
            x_y_second = [- x_second*cos_theta + y_second*sin_theta - ht_x,-x_second*sin_theta - y_second*cos_theta - ht_y]
            d_x = x_y_first[0] + x_y_second[0]
            d_y = x_y_first[1] + x_y_second[1]
            aux1=distance.euclidean((0, 0,), (d_x, d_y)) <= distance_thresh
            aux2=abs(d_theta) <= abs(angle_thresh)
            if flag_second[i] == 0.0 and flag_first[j] == 0.0 and aux1 and aux2 :
                flag_second[i] = 1.0
                count_matched =count_matched+1
                matched_minutiae.append(((x_first, y_first), (x_second, y_second)))
            j =j+ 1
        i =i+1
    
    return count_matched, i

def interactive_display(window_label, image):
	cv2.imshow(window_label, image)
	while True:
		key = cv2.waitKey(0) & 0xFF
		# wait for enter key to exit
		if key == 13:
			cv2.destroyAllWindows()
			break
	cv2.destroyAllWindows()

def match(first, second, img_1, img_2):
        matches, _ =  pairing(first, second, alignment_using_hough_transform(first,second))
        len1=len(first)
        len2=len(second)
        print("Fingerprint 1 has {} minutiae points".format(len1))
        print("Fingerprint 2 has {} minutiae points".format(len2))
        print("{} out of those match in between them".format(matches))
        print("{:.2f} percentage of minutiae matched".format(100*matches/min(len1, len2)))
        #interactive_display("Fingerprint 1 Minutiae", img_1)
        #interactive_display("Fingerprint 2 Minutiae", img_2)
if __name__=="__main__":
    path=os.getcwd()+"/testcases/"
    input=input("enter path name of testcases like 104:")
    path_fp_img_1=path+input+"_1.tif"
    path_fp_img_2=path+input+"_2.tif"
    #path_fp_img_1 = "G:/study/sem2/SIL775_BiometricSecurity/assignment/assignment1/Fingerprint-Verification-System-main/Fingerprint-Verification-System-main/testcases/104_1.tif"
    #path_fp_img_2 = "G:/study/sem2/SIL775_BiometricSecurity/assignment/assignment1/Fingerprint-Verification-System-main/Fingerprint-Verification-System-main/testcases/104_2.tif"
    img_1,minuate_1 = fingerprint_recognition(path_fp_img_1)
    img_2,minuate_2 = fingerprint_recognition(path_fp_img_2)
    match(minuate_1,minuate_2,img_1,img_2)